In [1]:
import pandas as pd
import QUANTAXIS as QA

QUANTAXIS>> Expand macros in /Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/macropy/core/hquotes.py
QUANTAXIS>> Finding macros in 'macropy.core.hquotes'
QUANTAXIS>> Importing macros from 'macropy.core.quotes' into 'macropy.core.hquotes'
QUANTAXIS>> Expand macros in /Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/macropy/core/quotes.py
QUANTAXIS>> Finding macros in 'macropy.core.quotes'
QUANTAXIS>> Expand macros in /Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/macropy/core/failure.py
QUANTAXIS>> Finding macros in 'macropy.core.failure'
QUANTAXIS>> Importing macros from 'macropy.core.hquotes' into 'macropy.core.failure'


no module named mssql


QUANTAXIS>> Expand macros in /Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/cffi/api.py
QUANTAXIS>> Finding macros in 'cffi.api'
QUANTAXIS>> Expand macros in /Volumes/QuantOsUSB/QuantOsUSB/kivy.app/Contents/Resources/venv/lib/python3.6/site-packages/scipy/__config__.py
QUANTAXIS>> Finding macros in 'scipy.__config__'


# 1.获取全市场股票列表

In [2]:
code=QA.QA_fetch_stock_list_adv().code.tolist()
code1=QA.QA_fetch_stock_list_adv().set_index('code')

# 2.获取全市场日线+1分钟线+5分钟线

In [ ]:
daydata=QA.QA_fetch_stock_day_adv(code,'2018-07-12','2018-07-13')

dailymin_data=QA.QA_fetch_stock_min_adv(code,'2018-07-13','2018-07-14')
dailymin5_data=QA.QA_fetch_stock_min_adv(code,'2018-07-13','2018-07-14','5min')

# 3.获取全市场版块

In [ ]:
block=QA.QA_fetch_stock_block_adv()

# 4.查看1分钟线单个bar涨幅超过1.8%的股票

In [ ]:
dailymin_data.fast_moving(0.018)

# 5. 查看5分钟线单个bar涨幅超过3%的股票

In [ ]:
fast_moving=dailymin5_data.fast_moving(0.03)

In [ ]:
fast_moving

# 6.涨停分析

## 6.1涨停在DataStruct是high_limit (惰性计算)

In [ ]:
#涨停

high_limit=daydata[daydata.close==daydata.high_limit]

## 6.2选择涨停的个股(select_code方法),并选取异动1.8%的股票

In [ ]:
data=dailymin_data.select_code(high_limit.code).fast_moving(0.018)

## 6.3更换名称

In [ ]:
yd=data.reset_index().rename(columns={0:'pct_change'})

## 6.4对异动的股票增加股票所属版块名, 股票名

datetime是发生异动的时间, code是股票名, pct_change是单个bar的涨幅 block是所属版块 name是股票名称

In [ ]:
yd=yd.assign(block=yd.code.apply(lambda x: block.get_code(x).block_name)).assign(name=yd.code.apply(lambda x: code1.loc[x]['name']))

In [ ]:
yd

## 6.5将异动的版块和时间对应起来

In [ ]:
res=pd.DataFrame([block.get_code(x).block_name for x in list(yd.code)], index=yd.set_index(['datetime','code']).index)

In [ ]:
res

# 7.查看异动个股所属版块

In [ ]:
block.get_code('002662').block_name

# 8.我们可以使用QA_DataStruct_series来加载异动series 方便我们查看

In [ ]:
mvs=QA.QA_DataStruct_Series(fast_moving)

In [ ]:
mvs

选取某一个股票

In [ ]:
mvs.select_code('000411').series

选取某一个时刻

In [ ]:
mvs.select_time('2018-07-13 09:35:00').series

In [ ]:
block.get_code(list(high_limit.code)[0])()